### Packages

In [1]:
import pandas as pd
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Utility for extracting HTML Table 

This is adapted from [Parsing HTML Tables in Python with BeautifulSoup and pandas](https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/)

In [2]:
class HTMLPageParser:

    soup = None

    def parse_url(self, url):
        paragraphs = []
        tables = []
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html')
        # parse paragraphs
        _paragraphs = soup.find_all('p')
        for p in _paragraphs:
            paragraphs.append(p.text)
        # parse tables
        _tables = soup.find_all('table')
        for t in _tables:
            table = self.parse_html_table(t)
            tables.append(table)
            
        return paragraphs, tables

    def parse_html_table(self, table):
        #print("new table")
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):

            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)

            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        df = pd.DataFrame() 
        try:                    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")

            columns = column_names if len(column_names) > 0 else range(0,n_columns)

            #print(n_rows, n_columns)
            df = pd.DataFrame(columns = columns,
                  index= range(0,n_rows))

            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')

                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1

            # Convert to float if possible
            for col in df:
                    df[col] = df[col]
        except Exception as ex:
            #print(ex)
            pass
        #df.head(10)
        return df


### Grab the historical data

Keep the historical data.

At a later stage, just download the daily new data and add to the time series.

In [3]:
url_base = "https://www.mai.gov.ro/informare-covid-19-grupul-de-comunicare-strategica-"
url_data =\
    [('2020-04-02',"2-aprilie-ora-13-00/"),
     ('2020-04-03',"3-aprilie-2020-ora-13-00/"),
     ('2020-04-04',"4-aprilie-2020-ora-13-00/"),
     ('2020-04-05',"5-aprilie-2020-ora-13-00/"),
     ('2020-04-06',"6-aprilie-2020-ora-13-00/"),
     ('2020-04-07', "6-aprilie-2020-ora-13-00-2/"),
     ('2020-04-08', "8-aprilie-2020-ora-13-00/"),
     ('2020-04-09', "9-aprilie-2020-ora-13-00/"),
     ('2020-04-10', "10-aprilie-2020-ora-13-00/"),
     ('2020-04-11', "11-aprilie-2020-ora-13-00/"),
     ('2020-04-12', "12-aprilie-2020-ora-13-00/"),
     ('2020-04-13', "13-aprilie-2020-ora-13-00/"),
     ('2020-04-14', "14-aprilie-2020-ora-13-00/"),
     ('2020-04-15', "15-aprilie-2020-ora-13-00/"),
     ('2020-04-16', "16-aprilie-2020-ora-13-00/"),
     ('2020-04-17', "16-aprilie-2020-ora-13-00-2/"),
     ('2020-04-18', "18-aprilie-2020-ora-13-00/")
    ]

In [4]:
import re
import numpy as np

def get_ati_patients(paragraphs):
    regex = "La ATI, în acest moment, sunt internați+(.[0-9]*)"
    for paragraph in paragraphs:
        group = 1
        pattern = re.compile(regex, re.UNICODE | re.IGNORECASE)
        results = [match.group(group) for match in pattern.finditer(paragraph)]
        if(results):
            return results[0]

In [5]:
def get_quarantine(paragraphs):
    regex = "în carantină instituționalizată sunt+(.[ 0-9.]*)"
    for paragraph in paragraphs:
        group = 1
        pattern = re.compile(regex, re.UNICODE | re.IGNORECASE)
        results = [match.group(group) for match in pattern.finditer(paragraph)]
        if(results):
            results = results[0]   
            return results    

In [6]:
def get_isolation(paragraphs):
    regex = "de persoane. Alte +(.[ 0-9.]*)"
    for paragraph in paragraphs:
        group = 1
        pattern = re.compile(regex, re.UNICODE | re.IGNORECASE)
        results = [match.group(group) for match in pattern.finditer(paragraph)]
        if(results):
            results = results[0]   
            return results    

In [7]:
def get_tests(paragraphs):
    regex = "Până la această dată, la nivel național, au fost prelucrate+(.[ 0-9.]*)"
    for paragraph in paragraphs:
        group = 1
        pattern = re.compile(regex, re.UNICODE | re.IGNORECASE)
        results = [match.group(group) for match in pattern.finditer(paragraph)]
        if(results):
            results = results[0]   
            return results        

In [8]:
hp = HTMLPageParser()
all_data_df = pd.DataFrame()
country_data_df = pd.DataFrame()
for current_date, current_url in url_data:
    compose_url = f"{url_base}{current_url}"
    paragraphs, tables = hp.parse_url(compose_url)
    
    # Process table data - to extract county-level data
    # retain only the first table
    payload_table = tables[0]
    print(f"current date: {current_date}, table rows: {payload_table.shape[0]}")
    payload_table['date'] = current_date
    #remove headers & footers
    payload_table = payload_table.iloc[1:]
    payload_table = payload_table.iloc[:-1]
    all_data_df = all_data_df.append(payload_table)
    
    # Process paragraph data - to extract country-level data
    ati = get_ati_patients(paragraphs)
    quarantine = get_quarantine(paragraphs)
    isolation = get_isolation(paragraphs)
    tests = get_tests(paragraphs)  
    country_data_df = country_data_df.append(pd.DataFrame({'date':current_date, 'ati': ati,\
                                    'quarantine': quarantine, 'isolation': isolation, 'tests': tests}, index=[0]))
    
all_data_df.columns = ['No', 'County', 'Confirmed', 'Date']

current date: 2020-04-02, table rows: 44
current date: 2020-04-03, table rows: 44
current date: 2020-04-04, table rows: 45
current date: 2020-04-05, table rows: 44
current date: 2020-04-06, table rows: 44
current date: 2020-04-07, table rows: 44
current date: 2020-04-08, table rows: 44
current date: 2020-04-09, table rows: 44
current date: 2020-04-10, table rows: 44
current date: 2020-04-11, table rows: 44
current date: 2020-04-12, table rows: 44
current date: 2020-04-13, table rows: 44
current date: 2020-04-14, table rows: 44
current date: 2020-04-15, table rows: 44
current date: 2020-04-16, table rows: 44
current date: 2020-04-17, table rows: 44
current date: 2020-04-18, table rows: 44


In [9]:
all_data_df.shape, all_data_df.columns

((715, 4), Index(['No', 'County', 'Confirmed', 'Date'], dtype='object'))

In [10]:
country_data_df

,date,ati,quarantine,isolation,tests
0,2020-04-02,78,,114.699,28.483
0,2020-04-03,83,,114.646,31.657
0,2020-04-04,119,,113.449,36.092
0,2020-04-05,141,,108.810,38.623
0,2020-04-06,179,23.849,106.463,40.987
0,2020-04-07,274,25.379,91.369,43.578
0,2020-04-08,162,25.556,89.976,47.207
0,2020-04-09,178,25.586,87.400,51.802
0,2020-04-10,183,25.226,77.317,55.430
0,2020-04-11,208,24.984,74.233,59.272


In [11]:
all_data_df.head()

,No,County,Confirmed,Date
1,1.,Alba,9,2020-04-02
2,2.,Arad,110,2020-04-02
3,3.,Argeș,10,2020-04-02
4,4.,Bacău,19,2020-04-02
5,5.,Bihor,39,2020-04-02


In [12]:
all_data_df.County.unique()

array(['Alba', 'Arad', 'Argeș', 'Bacău', 'Bihor', 'Bistrița-Năsăud',
       'Botoșani', 'Brașov', 'Brăila', 'Buzău', 'Caraș-Severin',
       'Călărași', 'Cluj', 'Constanța', 'Covasna', 'Dâmbovița', 'Dolj',
       'Galați', 'Giurgiu', 'Gorj', 'Harghita', 'Hunedoara', 'Ialomița',
       'Iași', 'Ilfov', 'Maramureș', 'Mehedinți', 'Mureș', 'Neamț', 'Olt',
       'Prahova', 'Satu Mare', 'Sălaj', 'Sibiu', 'Suceava', 'Teleorman',
       'Timiș', 'Tulcea', 'Vaslui', 'Vâlcea', 'Vrancea', 'Mun. București',
       '–'], dtype=object)

In [31]:
COUNTY_NUMBER = 42
try:
    assert (len(all_data_df.County.unique())== (COUNTY_NUMBER + 1)), "wrong County number"
except Exception as ex:
    print(f"Error: {ex}")

In [32]:
all_data_df.Confirmed.unique()

array(['9', '110', '10', '19', '39', '22', '40', '117', '11', '12', '13',
       '17', '105', '111', '34', '82', '7', '–', '100', '37', '54', '38',
       '107', '8', '27', '16', '6', '701', '98', '60', '505', '24', '44',
       '124', '26', '108', '90', '1', '45', '72', '46', '55', '150',
       '866', '126', '70', '544', '15', '128', '14', '28', '48', '127',
       '29', '121', '101', '23', '123', '50', '81', '64', '42', '57',
       '148', '25', '5', '967', '21', '136', '550', '33', '68', '131',
       '30', '114', '47', '56', '88', '78', '160', '49', '1.215', '176',
       '79', '552', '157', '52', '58', '84', '138', '31', '119', '32',
       '3', '63', '87', '89', '1.228', '181', '566', '161', '86', '156',
       '133', '158', '92', '99', '43', '182', '36', '69', '1.322', '191',
       '598', '83', '51', '35', '129', '174', '106', '41', '103', '196',
       '1.430', '227', '637', '162', '66', '76', '175', '62', '4', '212',
       '146', '202', '1.487', '252', '95', '659', '179', '

In [33]:
all_data_df.Date.unique()

array(['2020-04-02', '2020-04-03', '2020-04-04', '2020-04-05',
       '2020-04-06', '2020-04-07', '2020-04-08', '2020-04-09',
       '2020-04-10', '2020-04-11', '2020-04-12', '2020-04-13',
       '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17',
       '2020-04-18'], dtype=object)

### Replace '-' in County with 'Not identified'

In [34]:
all_data_df.loc[all_data_df['County']=='–', 'County'] = 'Not identified'

### Replace '-' in Confirmed with '0' and fix decimal point

In [35]:
# replace - in Confirmed
all_data_df.loc[all_data_df['Confirmed']=='–', 'Confirmed'] = 0
# fix decimal point
all_data_df['Confirmed'] = all_data_df['Confirmed'].astype(str)
all_data_df['Confirmed'] = all_data_df['Confirmed'].apply(lambda x: x.replace(".", ""))
all_data_df['Confirmed'] = all_data_df['Confirmed'].astype(int)

In [36]:
max(all_data_df.Confirmed), min(all_data_df.Confirmed)

(2019, 0)

## Global data, fix decimal point

In [37]:
country_data_df

,date,ati,quarantine,isolation,tests
0,2020-04-02,78,,114.699,28.483
0,2020-04-03,83,,114.646,31.657
0,2020-04-04,119,,113.449,36.092
0,2020-04-05,141,,108.810,38.623
0,2020-04-06,179,23.849,106.463,40.987
0,2020-04-07,274,25.379,91.369,43.578
0,2020-04-08,162,25.556,89.976,47.207
0,2020-04-09,178,25.586,87.400,51.802
0,2020-04-10,183,25.226,77.317,55.430
0,2020-04-11,208,24.984,74.233,59.272


In [38]:
# fix decimal point
country_data_df.loc[country_data_df['quarantine']==' ', 'quarantine'] = 0
for feature in ['ati', 'quarantine', 'isolation', 'tests']:
    country_data_df[feature] = country_data_df[feature].astype(str)
    country_data_df[feature] = country_data_df[feature].apply(lambda x: x.replace(".", ""))
    country_data_df[feature] = country_data_df[feature].astype(int)

In [39]:
all_data_df.County.unique()

array(['Alba', 'Arad', 'Argeș', 'Bacău', 'Bihor', 'Bistrița-Năsăud',
       'Botoșani', 'Brașov', 'Brăila', 'Buzău', 'Caraș-Severin',
       'Călărași', 'Cluj', 'Constanța', 'Covasna', 'Dâmbovița', 'Dolj',
       'Galați', 'Giurgiu', 'Gorj', 'Harghita', 'Hunedoara', 'Ialomița',
       'Iași', 'Ilfov', 'Maramureș', 'Mehedinți', 'Mureș', 'Neamț', 'Olt',
       'Prahova', 'Satu Mare', 'Sălaj', 'Sibiu', 'Suceava', 'Teleorman',
       'Timiș', 'Tulcea', 'Vaslui', 'Vâlcea', 'Vrancea', 'Mun. București',
       'Not identified'], dtype=object)

In [40]:
all_data_df.Confirmed.unique()

array([   9,  110,   10,   19,   39,   22,   40,  117,   11,   12,   13,
         17,  105,  111,   34,   82,    7,    0,  100,   37,   54,   38,
        107,    8,   27,   16,    6,  701,   98,   60,  505,   24,   44,
        124,   26,  108,   90,    1,   45,   72,   46,   55,  150,  866,
        126,   70,  544,   15,  128,   14,   28,   48,  127,   29,  121,
        101,   23,  123,   50,   81,   64,   42,   57,  148,   25,    5,
        967,   21,  136,  550,   33,   68,  131,   30,  114,   47,   56,
         88,   78,  160,   49, 1215,  176,   79,  552,  157,   52,   58,
         84,  138,   31,  119,   32,    3,   63,   87,   89, 1228,  181,
        566,  161,   86,  156,  133,  158,   92,   99,   43,  182,   36,
         69, 1322,  191,  598,   83,   51,   35,  129,  174,  106,   41,
        103,  196, 1430,  227,  637,  162,   66,   76,  175,   62,    4,
        212,  146,  202, 1487,  252,   95,  659,  179,  190,  152,   85,
        115,  234,   94, 1529,  259,  688,  229,   

In [41]:
for date in all_data_df.Date.unique():
    d_df = all_data_df.loc[all_data_df.Date==date]
    d_df.to_csv(os.path.join('ro_covid_19_daily_reports', f"{date}.csv"), index=False)

In [42]:
all_data_df.to_csv(os.path.join('ro_covid_19_time_series', "ro_covid_19_time_series.csv"), index=False)
country_data_df.to_csv(os.path.join('ro_covid_19_time_series', "ro_covid_19_country_data_time_series.csv"), index=False)

In [43]:
all_data_df.groupby('Date')['Confirmed'].sum()

Date
2020-04-02    2738
2020-04-03    3183
2020-04-04    3393
2020-04-05    3864
2020-04-06    4057
2020-04-07    4417
2020-04-08    4761
2020-04-09    5202
2020-04-10    5467
2020-04-11    5990
2020-04-12    6300
2020-04-13    6633
2020-04-14    6879
2020-04-15    7216
2020-04-16    7707
2020-04-17    8067
2020-04-18    8418
Name: Confirmed, dtype: int32